In [ ]:
library(opploansanalytics)
load.packages()

In [ ]:
getData = function () {

    df = queryReporting(
    "
    select
        c_adm.name as admethod
        , count(*) as qualified_count
        , max(c_adm.createddate) as admethod_createddate
        , case when c_adm_15.id notnull then 'New Admethod 15' else 'Old Admethod' end as admethod_recent
    from
        cloudlending.applications as c_app
        inner join
            public.all_allapps as p_ap
            on c_app.name = p_ap.loanid
            and left(p_ap.denygrp, 1) > '2'
            and p_ap.refi = 'N'
        inner join
            cloudlending.advertising_method as c_adm
            on c_app.advertising_method = c_adm.id
        left join
            (
                select
                    id
                from
                    cloudlending.advertising_method
                order by
                    createddate desc
                limit 15
            ) as c_adm_15
            on c_app.advertising_method = c_adm_15.id
    where
        c_app.createddate >= now()::date - '10 days'::interval
    group by
        1,4
    order by
        2 desc
    "
    )
    
    df %>%
    transmute(
        admethod,
        qualified_count,
        qualified_perc = qualified_count/sum(df$qualified_count),
        admethod_createddate,
        admethod_recent
    ) %>% 
    slice(
        1:25
    )
    
}

In [ ]:
getTrigger = function (df) {
    
    df %>%
        filter(
            admethod_recent == 'New Admethod 15'
        ) %>% 
        nrow() > 0
    
}

In [ ]:
sendEmail = function () {

    sender = 'jchang@opploans.com'
    recipients = c('jchang@opploans.com')
    send.mail(
        from = sender,
        to = recipients,
        subject = 'New Admethod Alert',
        body = "New Admethods Detected",
        attach.files = "..\\docs\\admethod-alert.csv",
        html = TRUE,
        inline = TRUE,
        smtp = list(
            host.name = 'smtp.gmail.com', 
            port = 465, 
            user.name = 'jchang@opploans.com',            
            passwd = Sys.getenv('R_GMAIL_2FAUTH'), 
            ssl = TRUE
        ),
        authenticate = TRUE,
        send = TRUE
    )
    
}

In [ ]:
executeAlert = function () {
    
    df = getData()
    
    if ( getTrigger(df) ) {
        
        df %>% write.csv("..\\docs\\admethod-alert.csv")
        sendEmail()
    }
    
}

In [ ]:
executeAlert()